In [1]:
!conda env list
#%pip install transformers                                                      
#%pip install soundfile   
#%pip install openai-whisper tqdm faster-whisper

# conda environments:
#
                         /home/kozi/Downloads/oobabooga_linux/installer_files/conda
                         /home/kozi/Downloads/oobabooga_linux/installer_files/env
base                     /home/kozi/anaconda3
nemo                     /home/kozi/anaconda3/envs/nemo
nemo2                 *  /home/kozi/anaconda3/envs/nemo2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# load NeMo models

import nemo.collections.asr as nemo_asr
import os
                                   

print("started loading models")
#quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")
asr_models = []
asr_models.append(nemo_asr.models.EncDecCTCModelBPE.restore_from("/home/kozi/Documents/stt_en_conformer_ctc_small.nemo")) #from_pretrained() vs restore_from()
asr_models.append(nemo_asr.models.EncDecCTCModelBPE.restore_from("/home/kozi/Documents/stt_en_conformer_ctc_large.nemo"))
print("finished loading models")

# source of models https://huggingface.co/models?library=nemo&sort=trending&search=nvidia%2Fstt_en_conformer or https://huggingface.co/nvidia/stt_en_conformer_ctc_small/tree/main
# or even https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/asr/models.html

#files = [os.path.join(data_dir, 'an4/wav/an4_clstk/mgah/cen2-mgah-b.wav')]
#for fname, transcription in zip(files, quartznet.transcribe(paths2audio_files=files)):
  #print(f"Audio in {fname} was recognized as: {transcription}")

In [ ]:
# transcribe with previously loaded models

def transcribe_files(dir):
    for root, dirs, files in os.walk(dir):
        for file in files:
            full_file_path = os.path.join(root, file)

            if file.endswith('.txt'):
                with open(full_file_path, 'r') as f:
                    for line in f.readlines():
                        print(line)
            if file.endswith('.flac'):
                for model in asr_models: 
                    transcribed = model.transcribe([full_file_path])
                    print(file, transcribed)

transcribe_files("/home/kozi/Documents/librispeech/test_other/test-other/367/130732/")


In [ ]:
# gpt-4 generated code to run facebook wav2vec models, not necessary

from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
    
# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# tokenize
input_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest").input_values  # Batch size 1

# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

In [ ]:
# download a subset of FLEURS if necessary

from datasets import load_dataset

#_FLEURS_LANG = sorted(["af_za", "am_et", "ar_eg", "as_in", "ast_es", "az_az", "be_by", "bn_in", "bs_ba", "ca_es", "ceb_ph", "cmn_hans_cn", "yue_hant_hk", "cs_cz", "cy_gb", "da_dk", "de_de", "el_gr", "en_us", "es_419", "et_ee", "fa_ir", "ff_sn", "fi_fi", "fil_ph", "fr_fr", "ga_ie", "gl_es", "gu_in", "ha_ng", "he_il", "hi_in", "hr_hr", "hu_hu", "hy_am", "id_id", "ig_ng", "is_is", "it_it", "ja_jp", "jv_id", "ka_ge", "kam_ke", "kea_cv", "kk_kz", "km_kh", "kn_in", "ko_kr", "ckb_iq", "ky_kg", "lb_lu", "lg_ug", "ln_cd", "lo_la", "lt_lt", "luo_ke", "lv_lv", "mi_nz", "mk_mk", "ml_in", "mn_mn", "mr_in", "ms_my", "mt_mt", "my_mm", "nb_no", "ne_np", "nl_nl", "nso_za", "ny_mw", "oc_fr", "om_et", "or_in", "pa_in", "pl_pl", "ps_af", "pt_br", "ro_ro", "ru_ru", "bg_bg", "sd_in", "sk_sk", "sl_si", "sn_zw", "so_so", "sr_rs", "sv_se", "sw_ke", "ta_in", "te_in", "tg_tj", "th_th", "tr_tr", "uk_ua", "umb_ao", "ur_pk", "uz_uz", "vi_vn", "wo_sn", "xh_za", "yo_ng", "zu_za"])
# ref. https://huggingface.co/datasets/google/xtreme_s/blob/main/xtreme_s.py line 105

fleurs_dataset_dirs = ["be_by", "de_de", "en_us", "fr_fr", "gl_es", "hr_hr", "hu_hu", "it_it", "pl_pl", "ru_ru", "uk_ua"]

for ds_string in ['de_de', 'en_us', 'fr_fr', 'hu_hu', 'ru_ru']:
    ds = load_dataset("google/xtreme_s", "fleurs.{}".format(ds_string))
    print(ds['train']['path'])

# or downloaded_model = load_dataset("/home/kozi/Documents/fleurs/ds_string/")

In [ ]:
# examine a fleurs dataset

from datasets import load_dataset

de = load_dataset("/home/kozi/Documents/fleurs/en_us/")
de['train']

In [6]:
# test transcription with the fleurs dataset

from datasets import load_dataset

print(de['train'][0]['audio']['array'])
print(de['train'][0]['audio']['path'])
wav = de['train'][0]['audio']['path']
flac = "/home/kozi/Documents/librispeech/test_other/test-other/367/293981/367-293981-0000.flac"
#asr_models[0].transcribe([flac])
asr_models[0].transcribe([wav, flac])


[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -3.15904617e-06
 -3.03983688e-06 -3.27825546e-06]
/home/kozi/Documents/fleurs/en_us/audio/train/10004088536354799741.wav


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['a tornado is a spinning column of very low pressure air which sues surrounding air inward and upward',
 'i swear answered sancho']

In [25]:
# example workflow for notebooks
# download librispeech dataset (please run from the terminal)
#   ref.: https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/asr/datasets.html
#!python3 /home/kozi/Documents/NeMo/scripts/dataset_processing/get_librispeech_data.py --data_root=data --data_set=test_other


# create manifest_json for the already downloaded FLEURS english test dataset:
# python3 /home/kozi/Documents/fleurs/fleurs_to_nemo_manifest.py /home/kozi/Documents/fleurs/en_us
# ... use whisper_test.ipynb (perhaps renamed since) to run whisper transcription for Fleurs ...
# ... use show_wer_cer_results.ipynb (perhaps renamed since) to calculate WER and CER for the manifests ...